- create .venv
- run `pip install -r requirements.txt`
- get API key from https://kisski.gwdg.de/leistungen/2-02-llm-service/
- create `.env` file with:
  > OPENAI_API_KEY = "YOUR-API-KEY"  
  > KISSKI_URL = "https://chat-ai.academiccloud.de/v1"

In [ ]:
# Load HTML File/s
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_dir="html").load_data()
print(f"Loaded {len(documents)} documents.")

# Chunk with SentenceSplitter
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200) 
# 1 token = 4 characters
nodes = splitter.get_nodes_from_documents(documents)
print(f"Generated {len(nodes)} chunks.") 


# Embed Chunks with HuggingFace
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")


Loaded 1 documents.
Generated 326 chunks.


In [8]:
# Create Index
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import VectorStoreIndex, StorageContext

faiss_index = faiss.IndexFlatL2(384)
vector_store = FaissVectorStore(faiss_index=faiss_index)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes,
    embed_model=embed_model,
    storage_context=storage_context,
)

# Save index
index.storage_context.persist(persist_dir="./faiss_index")
print("Index stored in ./faiss_index")

Index stored in ./faiss_index


In [9]:
# LLM

from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("KISSKI_URL")

if not api_key or not base_url:
    raise ValueError("Missing API key or URL.")

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

def ask_openai_llm(prompt: str) -> str:
    response = client.chat.completions.create(
        model="meta-llama-3.1-8b-instruct",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


In [13]:
# Question
import textwrap
from IPython.display import Markdown, display

while True:
    query = input("Enter your question (or type 'q'): ").strip()
    if query.lower() == 'q':
        print("Session ended.")
        break

    nodes = index.as_retriever().retrieve(query)
    context = "\n---\n".join([n.get_content() for n in nodes])
    full_prompt = f"""You are a helpful assistant. Use the following context to answer the question.

Context:
{context}

Question:
{query}"""

    answer = ask_openai_llm(full_prompt)
    print(f"\nQ:")
    display(Markdown(textwrap.dedent(query)))
    print("\nA:")
    display(Markdown(textwrap.dedent(answer)))
    print("___\n")


Q:


Where are the best curries in India?


A:


I'm happy to help, but the context provided doesn't seem to be related to the question about the best curries in India. The context appears to be about the impact of climate change on agriculture in Pacific island regions and the Caribbean.

If you could provide more context or information about what you're looking for, I'd be happy to try and assist you. Alternatively, if you'd like to discuss the impact of climate change on agriculture or other related topics, I'd be happy to chat with you about that as well.

___

Session ended.
